In [ ]:
import pprint
from rdflib import Namespace
from cmmn3 import parseModel, convertModel
from cmmn3 import reason

n3Path = "cmmn3/n3/"
xmlNs = {"cmmn": "http://www.omg.org/spec/CMMN/20151109/MODEL"}
xmlPath = "/Users/wvw/Dropbox/research/projects/Montfort PM/cmmn/cap-all.cmmn"
modelNs = Namespace("http://ontario.org/qbp/cap#")

## Model

In [32]:
itemObjs = parseModel(xmlPath, xmlNs)
# pprint.pprint(itemObjs)

modelPath = "ttl/cap/model.ttl"
convertModel(itemObjs, modelNs, modelPath)

## Log

In [33]:
from cmmn3 import convertLog

obsPath = "ttl/cap/obs.ttl"

log_stmts = convertLog("csv/log.csv", modelNs)
with open("ttl/cap/obs.ttl", 'w') as fh:
    fh.write(log_stmts[0])

## Reason

In [35]:
out = reason(modelPath, obsPath, n3Path, printerr=True)
print(out)

time: 0.08027262520045042
error: eye --quiet ttl/cap/model.ttl ttl/cap/obs.ttl cmmn3/n3/graph.n3 cmmn3/n3/state.n3 cmmn3/n3/workflow.n3 cmmn3/n3/run.n3 --nope --pass-only-new
EYE v11.23.2 (2025-12-09)
SWI-Prolog version 9.2.7
starting 29 [msec cputime] 32 [msec walltime]
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/ttl/cap/model.ttl SC=129
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/ttl/cap/obs.ttl SC=1
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/n3/graph.n3 SC=1
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/n3/state.n3 SC=3
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/n3/workflow.n3 SC=17
GET file:///Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/n3/run.n3 SC=8
networking 18 [msec cputime] 20 [msec walltime]
reasoning 14 [msec cputime] 14 [msec walltime]
2026-01-31T18:25:32.883Z in=159 out=12 ent=24 step=48 brake=4 inf=520999 sec=0.061 inf/sec=8540967


# Processed by EYE v11.23.2 (2025-12-09)
# eye --quiet ttl/cap/model.ttl ttl/cap/obs.ttl cmmn3/n3/graph.n3 cmmn3/n3/state.n3 cmm

# Show

In [39]:
from cmmn3 import parseOut

errors, finals = parseOut(out, itemObjs)

In [ ]:
showStates = []
extraMarkers = []

for item, ( id, state ) in finals.items():
    showStates.append(f"showState('{id}', '{state.lower()}', canvas, overlays);")
    
for item, ( id, error, itemState )  in errors.items():
    itemObj = itemObjs[item]
    match (error):
        case 'readyToCompleted':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
            for sentry in itemObj['sentries']['entry']:
                showStates.append(f"showState('{sentry['id']}', 'sentryViolated', canvas, overlays);")
                
        case 'inactiveToCompleted':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
        
        case 'mandatoryLastNotDone':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
            showStates.append(f"showState('{id}', 'firstSymbolViolated', canvas, overlays);")
            
        case 'nonRepetitiveMultipleCompleted':
            showStates.append(f"showState('{id}', 'error', canvas, overlays);")
            clsName = 'secondSymbolViolated' if (itemObj['mandatory']) else 'firstSymbolViolated'
            showStates.append(f"showState('{id}', '{clsName}', canvas, overlays);")
            extraMarkers.append(f"'{id}': {{ 'isRepeatable': true }}")
            
showStatesJs = "\n".join(showStates)
print(showStatesJs)
extraMarkersJs = ",".join(extraMarkers)
extraMarkersJs = f"window.extraMarkers = {{ { extraMarkersJs } }}"
print(extraMarkersJs)

# from IPython.display import display, HTML
# display(HTML(html))

with open("viewer/viewer-or.html", 'r') as fh:
    html = fh.read()
    
with open("viewer/viewer.html", 'w') as fh:
    html = html.replace("<extraMarkersPlaceholder>", extraMarkersJs)
    html = html.replace("<showStatesPlaceholder>", showStatesJs)
    fh.write(html)